In [1]:
import pandas as pd

In [3]:
# import hospitals to dataframes
public = pd.read_csv('4_LegalEntitySummaryPublicHospital.csv')
private = pd.read_csv('4_LegalEntitySummaryNGOHospital.csv')

In [4]:
# inspect both datasets
public.head()

,Premises Name,Certification Service Type,Service Types,Total Beds,Premises Website,Premises Address Other,Premises Address,Premises Address Suburb/Road,Premises Address Town/City,Premises Address Post Code,DHB Name,Certificate Name,Certification Period (Months),Certificate/License End Date,Current Auditor,Legal Name,Legal Entity Address Other,Legal Entity Address,Legal Entity Address Suburb/Road,Legal Entity Address Town/City,Legal Entity Address Post Code,Legal Entity Postal Address,Legal Entity Postal Address Suburb/Road,Legal Entity Postal Address Town/City,Legal Entity Postal Address Post Code,Legal Entity Website,Unnamed: 26
0,Auckland City Hospital,Public Hospital,"Childrens health, Maternity, Surgical, Medical",1165,NaN,NaN,2 Park Road,Grafton,Auckland,1023,Auckland District Health Board,Auckland District Health Board,No change,1-5-2023,The DAA Group Limited,Auckland District Health Board,"Level 51, Building 10, Greenlane Clinical Centre",214 Green Lane West,NaN,Auckland,1051,Private Bag 92024,NaN,Auckland,1142,http://www.adhb.govt.nz/,NaN
1,Auckland DHB X 3 Units - Mental Health,Public Hospital,Mental health,96,NaN,NaN,2 Park Road,Grafton,Auckland,1023,Auckland District Health Board,Auckland District Health Board,No change,1-5-2023,The DAA Group Limited,Auckland District Health Board,"Level 51, Building 10, Greenlane Clinical Centre",214 Green Lane West,NaN,Auckland,1051,Private Bag 92024,NaN,Auckland,1142,http://www.adhb.govt.nz/,NaN
2,Buchanan Rehabilitation Centre,Public Hospital,Mental health,40,NaN,NaN,27 Sutherland Road,Point Chevalier,Auckland,1025,Auckland District Health Board,Auckland District Health Board,No change,1-5-2023,The DAA Group Limited,Auckland District Health Board,"Level 51, Building 10, Greenlane Clinical Centre",214 Green Lane West,NaN,Auckland,1051,Private Bag 92024,NaN,Auckland,1142,http://www.adhb.govt.nz/,NaN
3,Greenlane Clinical Centre,Public Hospital,"Surgical, Medical",31,NaN,NaN,214 Green Lane West,Epsom,Auckland,1051,Auckland District Health Board,Auckland District Health Board,No change,1-5-2023,The DAA Group Limited,Auckland District Health Board,"Level 51, Building 10, Greenlane Clinical Centre",214 Green Lane West,NaN,Auckland,1051,Private Bag 92024,NaN,Auckland,1142,http://www.adhb.govt.nz/,NaN
4,Tauranga Hospital,Public Hospital,"Psychogeriatric, Geriatric, Mental health, Chi...",360,NaN,NaN,829 Cameron Road,Tauranga South,Tauranga,3112,Bay of Plenty District Health Board,Bay of Plenty District Health Board,36,3-10-2023,The DAA Group Limited,Bay of Plenty District Health Board,NaN,829 Cameron Road,NaN,Tauranga,3112,Private Bag 12024,NaN,Tauranga,3143,NaN,NaN


In [5]:
private.head()

,Premises Name,Certification Service Type,Service Types,Total Beds,Premises Website,Premises Address Other,Premises Address,Premises Address Suburb/Road,Premises Address Town/City,Premises Address Post Code,DHB Name,Certificate Name,Certification Period (Months),Certificate/License End Date,Current Auditor,Legal Name,Legal Entity Address Other,Legal Entity Address,Legal Entity Address Suburb/Road,Legal Entity Address Town/City,Legal Entity Address Post Code,Legal Entity Postal Address,Legal Entity Postal Address Suburb/Road,Legal Entity Postal Address Town/City,Legal Entity Postal Address Post Code,Legal Entity Website,Unnamed: 26
0,Bowen Hospital,NGO Hospital,"Surgical, Medical",44,NaN,NaN,98 Churchill Drive,Crofton Downs,Wellington,6035,Capital and Coast District Health Board,Acurity Health Group Limited - Bowen Hospital,No change,24-5-2023,The DAA Group Limited,Acurity Health Group Limited,NaN,30 Florence Street,NaN,Wellington,6021,Private Bag 7909,NaN,Wellington,6242.0,NaN,NaN
1,Royston Hospital,NGO Hospital,"Surgical, Medical",33,NaN,NaN,500 Southland Road,NaN,Hastings,4122,Hawke's Bay District Health Board,Acurity Health Group Limited - Royston Hospital,36,10-5-2023,The DAA Group Limited,Acurity Health Group Limited,NaN,30 Florence Street,NaN,Wellington,6021,Private Bag 7909,NaN,Wellington,6242.0,NaN,NaN
2,Wakefield Hospital,NGO Hospital,"Surgical, Medical",71,NaN,NaN,30 Florence Street,Newtown,Wellington,6021,Capital and Coast District Health Board,Acurity Health Group Limited - Wakefield Hospital,No change,27-5-2022,The DAA Group Limited,Acurity Health Group Limited,NaN,30 Florence Street,NaN,Wellington,6021,Private Bag 7909,NaN,Wellington,6242.0,NaN,NaN
3,Anglesea Hospital,NGO Hospital,Surgical,17,http://www.angleseahospital.co.nz/,Knox Clinic,Knox Street,Hamilton Central,Hamilton,3204,Waikato District Health Board,Anglesea Hospital Limited - Anglesea Hospital,No change,3-6-2022,Health and Disability Auditing New Zealand Lim...,Anglesea Hospital Limited,Knox Clinic,Knox Street,NaN,Hamilton,3204,PO Box 228,NaN,Hamilton,3240.0,NaN,NaN
4,Arohanui Hospice Service Trust,NGO Hospital,Medical,11,http://www.arohanuihospice.org.nz/,Arohanui Hospice,1 Heretaunga Street,NaN,Palmerston North,4414,MidCentral District Health Board,Arohanui Hospice Service Trust - Arohanui Hosp...,No change,13-7-2022,The DAA Group Limited,Arohanui Hospice Service Trust,NaN,1 Heretaunga Street,NaN,Palmerston North,4414,PO Box 5349,NaN,Palmerston North,4441.0,NaN,NaN


In [6]:
print(len(public))
print(len(private))

83
72


There are 83 public hospitals and 72 private. Both dataset seem to have the same columns, hence I should be able to concatenate them into a single list.

In [7]:
hospitals = pd.concat([public, private])
print(len(hospitals))

155


As expected, the concatenated list contains 155 records.

In [8]:
hospitals.head(1)

,Premises Name,Certification Service Type,Service Types,Total Beds,Premises Website,Premises Address Other,Premises Address,Premises Address Suburb/Road,Premises Address Town/City,Premises Address Post Code,DHB Name,Certificate Name,Certification Period (Months),Certificate/License End Date,Current Auditor,Legal Name,Legal Entity Address Other,Legal Entity Address,Legal Entity Address Suburb/Road,Legal Entity Address Town/City,Legal Entity Address Post Code,Legal Entity Postal Address,Legal Entity Postal Address Suburb/Road,Legal Entity Postal Address Town/City,Legal Entity Postal Address Post Code,Legal Entity Website,Unnamed: 26
0,Auckland City Hospital,Public Hospital,"Childrens health, Maternity, Surgical, Medical",1165,NaN,NaN,2 Park Road,Grafton,Auckland,1023,Auckland District Health Board,Auckland District Health Board,No change,1-5-2023,The DAA Group Limited,Auckland District Health Board,"Level 51, Building 10, Greenlane Clinical Centre",214 Green Lane West,NaN,Auckland,1051,Private Bag 92024,NaN,Auckland,1142.0,http://www.adhb.govt.nz/,NaN


In [9]:
# Next, I will only keep the relevant columns
hospitals = hospitals[['Premises Name', 'Certification Service Type', ' Premises Address',
                       ' Premises Address Suburb/Road', ' Premises Address Town/City', 
                       ' Premises Address Post Code', ' DHB Name']]
# and rename them to match the format
hospitals = hospitals.rename(columns={'Premises Name': 'name', ' Premises Address': 'address',
                                      ' Premises Address Suburb/Road': 'city',
                                      ' Premises Address Town/City': 'state',
                                      ' Premises Address Post Code': 'zip',
                                      'Certification Service Type': 'type',
                                      ' DHB Name': 'county'})
# and add the missing columns
hospitals[''] = hospitals.index
hospitals['id'] = hospitals.index
hospitals['phone'] = ''
hospitals['ownership'] = ''
hospitals['emergency'] = ''
hospitals['quality'] = ''
# and reaarange them
hospitals = hospitals[['id', 'name', 'address', 'city', 'state', 'zip', 'county',
                       'phone', 'type', 'ownership', 'emergency', 'quality']]
# check that I have what I expected
hospitals.head()

,id,name,address,city,state,zip,county,phone,type,ownership,emergency,quality
0,0,Auckland City Hospital,2 Park Road,Grafton,Auckland,1023,Auckland District Health Board,,Public Hospital,,,
1,1,Auckland DHB X 3 Units - Mental Health,2 Park Road,Grafton,Auckland,1023,Auckland District Health Board,,Public Hospital,,,
2,2,Buchanan Rehabilitation Centre,27 Sutherland Road,Point Chevalier,Auckland,1025,Auckland District Health Board,,Public Hospital,,,
3,3,Greenlane Clinical Centre,214 Green Lane West,Epsom,Auckland,1051,Auckland District Health Board,,Public Hospital,,,
4,4,Tauranga Hospital,829 Cameron Road,Tauranga South,Tauranga,3112,Bay of Plenty District Health Board,,Public Hospital,,,


Finally, I need to obtain coordinates for each hospital. I can use GeoPy again for this.

In [10]:
# install needed libraries
!pip install geopy 
!pip install Nominatim

  Created wheel for Nominatim: filename=nominatim-0.1-py3-none-any.whl size=2363 sha256=95d2a3168389a83064d12e61226b6ab9e24ceb9e0a836e8df9d113cd31a0b728
  Stored in directory: /root/.cache/pip/wheels/37/00/9e/d904c390bfb174830ad3dcfd62af5544cee7d88bed4f8acedd
Successfully built Nominatim


In [11]:
# initiate geolocator
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Your_Name")

In [12]:
# set up empty columns for latitude and longitude
hospitals['LAT'] = ''
hospitals['LON'] = ''

In [13]:
# iterate over all hospitals
for index, row in hospitals.iterrows():
    # get the location
    # a few of the calues were not strings, and hence needed to be cast as such
    address = ' '.join([str(row['address']), str(row['city']), row['state'], str(row['zip']), "New Zealand"])
    location = geolocator.geocode({address})
    # if location has been found
    if location != None:
      # add the coordinates to the dataframe
      hospitals.at[index,'LAT'] = location.latitude
      hospitals.at[index,'LON'] = location.longitude

In [14]:
# check if all locations were found
hospitals.loc[hospitals['LAT'] == ''].count()

id           5
name         5
address      5
city         3
state        5
zip          5
county       5
phone        5
type         5
ownership    5
emergency    5
quality      5
LAT          5
LON          5
dtype: int64

5 hospitals were not found.
I can amend them manually.

In [15]:
for index, row in hospitals.iterrows():
  # if no coordinates have been found
  if row['LAT'] == '':
    print(f"The name of the hospital is: {row['name']}")
    address = ' '.join([str(row['address']), str(row['city']), row['state'], str(row['zip']), "New Zealand"])
    print(f"The address is: {address}")
    lat = input('Latitude:')
    lon = input('Longitude:')
    # add the found data to the relevant columns
    hospitals.at[index,'LAT'] = lat
    hospitals.at[index,'LON'] = lon

The name of the hospital is: Alexandra Hospital
The address is: Gilbert Street Richmond Nelson 7020 New Zealand


KeyboardInterrupt: ignored

In [16]:
# check again if all locations have now been completed
hospitals.loc[hospitals['LAT'] == ''].count()

id           5
name         5
address      5
city         3
state        5
zip          5
county       5
phone        5
type         5
ownership    5
emergency    5
quality      5
LAT          5
LON          5
dtype: int64

In [17]:
# I noticed that my index contains duplicates, I need to reset it.
hospitals = hospitals.reset_index()
hospitals['_'] = hospitals.index
hospitals['id'] = hospitals.index
hospitals = hospitals[['_', 'id', 'name', 'address', 'city', 'state', 'zip', 'county', 'phone',
       'type', 'ownership', 'emergency', 'quality', 'LAT', 'LON']]

## Save as csv


In [18]:
hospitals.to_csv('hospitals.csv', index=False)